In [1]:
from collections import OrderedDict
import dotenv
import random
import numpy as np
import json
import pickle
import os
import numpy as np
import pandas as pd
import itertools
import openai
import schema
env_file = '.env'
dotenv.load_dotenv(env_file, override=True)
client = openai.OpenAI(api_key=os.getenv("CORRELL_API_KEY"))
from patient_prompt import prompt as pp
from doctor_prompt_structured import prompt as dp

patient_profiles = pickle.load(open('patient_profiles.pkl', 'rb'))
threshold = 0.8
steps = 5
num_profiles = 1 # how many interactions to run out of 240 profiles
patient_profiles: dict[int, schema.Profile] = OrderedDict(itertools.islice(patient_profiles.items(), num_profiles))

In [ ]:
# Function to call OpenAI API
def call_openai_doctor(messages: list[schema.Message]) -> schema.DoctorResponse:
    response = client.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=messages,
        temperature=0.4,
        response_format=schema.DoctorResponse
    )
    return response.choices[0].message.parsed

def call_openai(messages: list[schema.Message]) -> str:
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        temperature=0.7
    )
    return response.choices[0].message.content

doctor_histories: dict[int, list[schema.Message]] = {}
patient_histories: dict[int, list[schema.Message]] = {}

# Iterate through patient profiles
for i, profile in patient_profiles.items():
    doctor_config = {
        "gender": profile["gender"],
        "ethnicity": profile["ethnicity"],
        "confidence_threshold": threshold,
        "interaction_steps": steps
    }

    # Initialize metadata
    metadata = profile['interaction_metadata']
    metadata.update({
        "diagnosis": None,
        "diagnosis_success": False,
        "interaction_duration": 0,
        "num_symptoms_recovered": 0,
        "confidence_history": []
    })
    
    # Format prompts
    pp_copy = pp.format(**profile)
    dp_copy = dp.format(**doctor_config)
    
    # Initialize conversation histories
    doctor_history: list[schema.Message] = [{"role": "system", "content": dp_copy}]
    patient_history: list[schema.Message] = [{"role": "system", "content": pp_copy}]

    patient_reply = ''

    next_response_is_last = False
    
    # Run interaction loop
    for step in range(steps):
        metadata["interaction_duration"] += 1
        
        # Update doctor conversation history
        doctor_history.append({"role": "user", "content": "" if step == 0 else patient_reply})

        # Doctor response
        doctor_response = call_openai_doctor(doctor_history)
        
        # Update doctor conversation history with its reply
        doctor_history.append({"role": "assistant", "content": doctor_response.model_dump_json()})
        
        if len(doctor_response.diagnosis_rankings):
            diagnosis = max(doctor_response.diagnosis_rankings,  key=lambda x: x.confidence)
            metadata['diagnosis'] = diagnosis.diagnosis
            metadata['confidence_history'].append(diagnosis.confidence)
        # TODO: Counting could use improvement
        metadata['num_symptoms_recovered'] += 1 if any(symptom in patient_reply for symptom in doctor_response.symptoms_to_verify_or_refute) else 0
        
        if next_response_is_last:
            break
        elif metadata["confidence_history"] and metadata["confidence_history"][-1] >= threshold:
            metadata["diagnosis_success"] = True
            next_response_is_last = True

        # Update the patient history
        patient_history.append({"role": "user", "content": doctor_response.response_to_patient})

        # Patient response
        patient_reply = call_openai(patient_history + [{"role": "user", "content": doctor_response.response_to_patient}])
        
        # Update patient conversation history
        patient_history.append({"role": "assistant", "content": patient_reply})
    
    # Store updated metadata
    profile['interaction_metadata'] = metadata

    # Store conversation histories
    doctor_histories[i] = doctor_history
    patient_histories[i] = patient_history

ChatCompletionMessage(content="Hi! Well, I've got this spot on my skin that's bigger than the eraser on a pencil, and it feels like it's still growing. It also feels thicker or harder than the rest of my skin, like there's something under there. Plus, it looks kinda like a smooth, shiny pimple, but it never pops or goes away. That's what I've been worried about.", refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)
ChatCompletionMessage(content="I wouldn't say it's painful, but it does feel uncomfortable sometimes. And yeah, it’s super itchy sometimes, like a weird sensation under my skin. As for color, it's got a bunch of different colors—like brown, black, maybe even some red or blue mixed in.", refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)
ChatCompletionMessage(content="Yes, I swear this mole or bump got bigger really fast, like in just a few weeks. I've had it for a while now, but it feels like the changes have been pr

In [3]:
doctor_history

[{'role': 'system',
  'content': "\nYou are a doctor diagnosing a patient by asking questions and gathering information. Follow structured response tools.\n\nThe following information about the patient is known:\ngender: man\nethnicity: asian\n\nThere are two other parameters relevant to the following rules:\ninteraction_steps: 5\nconfidence_threshold: 0.8\n\nFor each of part of the structured response, refer to the following additional guidance.\n\nRules:\n---\n1. free_form_thoughts: str\n   - In free-form thoughts, synthesize the internal information and patient replies concisely to prioritize further questioning and diagnosis.\n   - Leverage the free_form_thoughts from previous interactions to inform the current interaction.\n   - Avoid excessive length to maintain a structured thought process and guide the following structured generations.\n2. known_symptoms: list[str]\n   - Broadly, use the patient information to inform further analysis and questioning.\n   - Match colloquial symp

In [4]:
metadata

{'diagnosis': 'Melanoma',
 'diagnosis_success': True,
 'interaction_duration': 5,
 'num_symptoms_recovered': 0,
 'confidence_history': [0.7, 0.85]}

In [5]:
patient_history

[{'role': 'system',
  'content': '\nYou are a patient interacting with a doctor to discuss your symptoms. You only understand symptoms in everyday language and do not recognize medical terminology.\n\n### **Guidelines for Responding:**\n1. **Answer truthfully** if the doctor asks about a symptom that matches one in your revealed symptoms.\n2. **Deny knowledge** of symptoms you haven’t noticed.\n3. **Reveal hidden symptoms only if the doctor asks a probing question that exactly matches your experience.**  \n   - Example: If you have been experiencing nausea (hidden), and the doctor asks, “Do you feel nauseous?”—you should confirm it.\n4. **Ask for clarification** if the doctor’s question uses unfamiliar medical terms.\n5. **Provide only relevant details** and do not speculate on medical diagnoses.\n6. When initially asked how the patient is feelilng, the patient should describe their revealed symptoms to the doctor using the colloquial descriptions provided.\n\n---\n\n### **Your Profile

In [6]:
doctor_history

[{'role': 'system',
  'content': "\nYou are a doctor diagnosing a patient by asking questions and gathering information. Follow structured response tools.\n\nThe following information about the patient is known:\ngender: man\nethnicity: asian\n\nThere are two other parameters relevant to the following rules:\ninteraction_steps: 5\nconfidence_threshold: 0.8\n\nFor each of part of the structured response, refer to the following additional guidance.\n\nRules:\n---\n1. free_form_thoughts: str\n   - In free-form thoughts, synthesize the internal information and patient replies concisely to prioritize further questioning and diagnosis.\n   - Leverage the free_form_thoughts from previous interactions to inform the current interaction.\n   - Avoid excessive length to maintain a structured thought process and guide the following structured generations.\n2. known_symptoms: list[str]\n   - Broadly, use the patient information to inform further analysis and questioning.\n   - Match colloquial symp

In [7]:
import save

save.save_history(patient_profiles[0], patient_history, doctor_history)

In [8]:
patient_history

[{'role': 'system',
  'content': '\nYou are a patient interacting with a doctor to discuss your symptoms. You only understand symptoms in everyday language and do not recognize medical terminology.\n\n### **Guidelines for Responding:**\n1. **Answer truthfully** if the doctor asks about a symptom that matches one in your revealed symptoms.\n2. **Deny knowledge** of symptoms you haven’t noticed.\n3. **Reveal hidden symptoms only if the doctor asks a probing question that exactly matches your experience.**  \n   - Example: If you have been experiencing nausea (hidden), and the doctor asks, “Do you feel nauseous?”—you should confirm it.\n4. **Ask for clarification** if the doctor’s question uses unfamiliar medical terms.\n5. **Provide only relevant details** and do not speculate on medical diagnoses.\n6. When initially asked how the patient is feelilng, the patient should describe their revealed symptoms to the doctor using the colloquial descriptions provided.\n\n---\n\n### **Your Profile

In [9]:
metadata

{'diagnosis': 'Melanoma',
 'diagnosis_success': True,
 'interaction_duration': 5,
 'num_symptoms_recovered': 0,
 'confidence_history': [0.7, 0.85]}